# Laboratorio 2: Data Understanding

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Machine Learning Operations** 

## Integrantes

- Arturo Argueta - 21527 
- Edwin de León - 22809 
- Diego Leiva - 21752 
- Pablo Orellana - 21970

## Librería

In [1]:
import pandas as pd
import numpy as np

## Carga de dataset

In [2]:
categoria_df = pd.read_csv('data/categoria.csv', encoding='latin1')
cliente_df = pd.read_csv('data/cliente.csv', encoding='latin1')
events_df = pd.read_csv('data/events.csv', encoding='latin1')
marca_df = pd.read_csv('data/marca.csv', encoding='latin1')
producto_df = pd.read_csv('data/producto.csv', encoding='latin1')

In [3]:
categoria_df.head()

,id,categoria
0,1,SCOTCH WHISKIES
1,2,STRAIGHT BOURBON WHISKIES
2,3,BLENDED WHISKIES
3,4,IMPORTED DRY GINS
4,5,DECANTERS & SPECIALTY PACKAGES


In [4]:
cliente_df.head()

,id,nombre,apellido,nacimiento,genero,empresa,idioma,nit,puesto,ciudad,correo,telefono
0,599528.0,Samuel,Ward,4/6/89,Male,Yakijo,Marathi,411-44-7088,Geologist IV,Wangjing,sward0@tamu.edu,86-(786)608-5061
1,121688.0,Willie,Gonzales,6/29/72,Male,Zoonoodle,Maltese,701-87-7540,Programmer III,El Corozo,wgonzales1@apache.org,58-(265)301-3397
2,552148.0,Betty,Spencer,9/2/83,Female,Youtags,Dhivehi,373-88-4503,Engineer III,Jinhua,bspencer2@shutterfly.com,86-(195)193-9042
3,102019.0,Beverly,Jordan,1/15/72,Female,Fivespan,Hindi,447-80-5871,Software Test Engineer IV,Salvacion,bjordan3@vimeo.com,63-(652)708-7688
4,189384.0,Cynthia,Flores,2/6/71,Female,Jabbersphere,Tsonga,803-60-8259,Speech Pathologist,Khorol,cflores4@webeden.co.uk,380-(373)389-5435


In [5]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [6]:
marca_df.head()

,id,marca
0,1,Diageo Americas
1,2,Heaven Hill Brands
2,3,"Sazerac Co., Inc."
3,4,Sage Beverages
4,5,MHW Ltd


In [7]:
producto_df.head()

,id,categoria_id,nombre,marca_id,volumen,precio
0,356475,9.0,Crown Royal Honey,1.0,750,22.49
1,15335,9.0,Crown Royal Regal Apple Mini,1.0,300,11.03
2,81345,9.0,Crown Royal Regal Apple,1.0,200,7.08
3,150318,9.0,Crown Royal Xr Canadian Whiskey,1.0,750,98.99
4,310791,9.0,Crown Royal Canadian Whisky Mini,1.0,300,11.03
